In [16]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup
from jim_decoding_functions import *


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin', 'C:/Users/j

In [2]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [3]:
stimulus_experiment_conditions = {
    "Stimulus/i25.0/s25.0": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/s75.0": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/s25.0": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/s75.0": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i25.0/r25.0": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/r75.0": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/r25.0": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/r75.0": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/s25.0": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/s75.0": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/s25.0": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/s75.0": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/r25.0": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/r75.0": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/r25.0": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/r75.0": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response/i25.0/s25.0": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i25.0/s75.0": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i75.0/s25.0": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i75.0/s75.0": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i25.0/r25.0": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i25.0/r75.0": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/i75.0/r25.0": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i75.0/r75.0": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c25.0/s25.0": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c25.0/s75.0": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c75.0/s25.0": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c75.0/s75.0": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c25.0/r25.0": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c25.0/r75.0": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c75.0/r25.0": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c75.0/r75.0": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [19]:
class GroupData:
    data = LabeledArray([])

    def __init__(self, subject_epochs: dict[str, mne.Epochs]):
        new_data = {}
        for sub, epochs in subject_epochs.items():
            new_data[sub] = LabeledArray.from_signal(epochs).to_dict()
        
        combined_sub_channel_data = combine(new_data, (0,2))

        self.data = LabeledArray.from_dict(combined_sub_channel_data)
        self.subjects = set(ch.split("-")[0] for ch in self.data.labels[1])
    
    # @property
    # def subjects(self):
    #     return set(ch.split("-")[0] for ch in self.data.labels[1])
    
seeg = mne.io.read_raw(mne.datasets.misc.data_path() /
                       'seeg' / 'sample_seeg_ieeg.fif')
on1 = trial_ieeg(seeg, 'Fixation', (-1, 1))

group_data = GroupData({'test1': on1})

subjects = group_data.subjects
print(subjects)

{'test1'}


In [18]:
group_data.data

array([[[ 0.00805439,  0.0080528 ,  0.00805067, ...,  0.00804961,
          0.00806237,  0.00805599],
        [-0.00692292, -0.0069277 , -0.00692664, ..., -0.00684901,
         -0.00683679, -0.00684263],
        [ 0.00182156,  0.0018189 ,  0.00181996, ...,  0.00191673,
          0.00193161,  0.00192577],
        ...,
        [-0.00611583, -0.00611743, -0.00611796, ..., -0.00609031,
         -0.0060733 , -0.00607436],
        [-0.00639603, -0.00639922, -0.0063955 , ..., -0.00640347,
         -0.00639443, -0.00640134],
        [ 0.00063113,  0.00063805,  0.0006524 , ...,  0.00071301,
          0.00072684,  0.00072524]]])
labels(['Fixation']
       ['test1-LENT 1', 'test1-LENT 2', 'test1-LENT 3', 'test1-LENT 4', 'test1-LENT 5', 'test1-LENT 6', 'test1-LENT 7', 'test1-LAMY 1', 'test1-LAMY 2', 'test1-LAMY 3', 'test1-LAMY 4', 'test1-LAMY 5', 'test1-LAMY 6', 'test1-LAMY 7', 'test1-LPHG 1', 'test1-LPHG 2', 'test1-LPHG 3', 'test1-LPHG 4', 'test1-LPHG 5', 'test1-LPHG 6', 'test1-LPHG 7', 'test1-LP

In [106]:
task='GlobalLocal'

conditions = stimulus_experiment_conditions # toggle

if conditions == stimulus_conditions or stimulus_experiment_conditions:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
elif conditions == response_conditions or response_experiment_conditions:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
else:
    raise ValueError("Unknown condition type.")

condition_names = list(conditions.keys()) # get the condition names as a list

# This breaks if just_HG_ev1_rescaled is set to False currently. Fix this! 8/11
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus/i25.0/s25.0 with parameters: {'BIDS_events': 'Stimulus/i25.0/s25.0', 'congruency': 'i', 'congruencyProportion': '75%', 'switchType': 's', 'switchProportion': '25%'}
  Loading condition: Stimulus/i25.0/s75.0 with parameters: {'BIDS_events': 'Stimulus/i25.0/s75.0', 'congruency': 'i', 'congruencyProportion': '75%', 'switchType': 's', 'switchProportion': '75%'}
  Loading condition: Stimulus/i75.0/s25.0 with parameters: {'BIDS_events': 'Stimulus/i75.0/s25.0', 'congruency': 'i', 'congruencyProportion': '25%', 'switchTy

In [172]:
subjects_mne_objects

{'D0057': {'Stimulus/i25.0/s25.0': {'HG_ev1_rescaled': <EpochsFIF | 7 events (all good), -1 – 1.5 s (baseline off), ~6.2 MB, data loaded,
    'Stimulus/i25.0/s25.0/BigLetterh/SmallLetters/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount265.0/BlockTrialCount41.0/ReactionTime1566.0/Accuracy1.0/D57': 1
    'Stimulus/i25.0/s25.0/BigLetterh/SmallLetters/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount306.0/BlockTrialCount82.0/ReactionTime1083.0/Accuracy1.0/D57': 1
    'Stimulus/i25.0/s25.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount229.0/BlockTrialCount5.0/ReactionTime1133.0/Accuracy1.0/D57': 1
    'Stimulus/i25.0/s25.0/BigLetterh/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount263.0/BlockTrialCount39.0/ReactionTime1383.0/Accuracy1.0/D57': 1
    'Stimulus/i25.0/s25.0/BigLetterh/SmallLette

In [107]:
condition_names = list(conditions.keys()) # get the condition names as a list
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus/i25.0/s25.0',
 'Stimulus/i25.0/s75.0',
 'Stimulus/i75.0/s25.0',
 'Stimulus/i75.0/s75.0',
 'Stimulus/i25.0/r25.0',
 'Stimulus/i25.0/r75.0',
 'Stimulus/i75.0/r25.0',
 'Stimulus/i75.0/r75.0',
 'Stimulus/c25.0/s25.0',
 'Stimulus/c25.0/s75.0',
 'Stimulus/c75.0/s25.0',
 'Stimulus/c75.0/s75.0',
 'Stimulus/c25.0/r25.0',
 'Stimulus/c25.0/r75.0',
 'Stimulus/c75.0/r25.0',
 'Stimulus/c75.0/r75.0']

get significant channels

In [108]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [109]:
# Assuming you have a list of subjects
root_dir = rf"C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs"

In [110]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'occ_filtered': [],
    'occ_best_filtered': []
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi, = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

let's manually make the occ filtered sig electrodes

In [111]:
sig_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

and now make the best of the best

In [112]:
sig_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

In [113]:
# Specify the file path where you want to save the dictionary
save_path = 'sig_electrodes_per_subject_roi.json'

# Use json to save the dictionary
with open(save_path, 'w') as file:
    json.dump(sig_electrodes_per_subject_roi, file, indent=4)

print(f"Dictionary saved to {save_path}")

Dictionary saved to sig_electrodes_per_subject_roi.json


get electrode counts for each roi

In [114]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 105
Total number of significant lpfc electrodes across all subjects: 46
Total number of v1 electrodes across all subjects: 17
Total number of significant v1 electrodes across all subjects: 12
Total number of occ electrodes across all subjects: 57
Total number of significant occ electrodes across all subjects: 41
Total number of occ_filtered electrodes across all subjects: 0
Total number of significant occ_filtered electrodes across all subjects: 23
Total number of occ_best_filtered electrodes across all subjects: 0
Total number of significant occ_best_filtered electrodes across all subjects: 10


check if any subjects have a weird sampling rate

In [115]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [116]:
# Example call to the function (you need to replace the arguments with actual data)
dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

subjects:  ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
num channels:  46
Processing D0057 for Stimulus/i25.0/s25.0 in lpfc
Number of trials for D0057 in Stimulus/i25.0/s25.0: 7
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/i25.0/s75.0 in lpfc
Number of trials for D0057 in Stimulus/i25.0/s75.0: 14
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/i75.0/s25.0 in lpfc
Number of trials for D0057 in Stimulus/i75.0/s25.0: 22
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/i75.0/s75.0 in lpfc
Number of trials for D0057 in Stimulus/i75.0/s75.0: 63
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/i25.0/r25.0 in lpfc
Number of trials for D0057 in Stimulus/i25.0/r25.0: 20
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/i25.0/r75.0 in lpfc
Number of trials for D0057 in Stimulus/i25.0/r75.0: 2
sub:  D0057
channel indices:  [0, 1]
Processing D

make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for this condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for this condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for this condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.


In [117]:
import numpy as np

def put_data_in_labeled_array_per_roi_subject(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi):
    """
    Organize the MNE data into separate LabeledArray for each ROI and subject, with NaN padding for trial dimension.
    Concatenates subject data along the channels axis.
    
    Parameters:
    - subjects_mne_objects: Dictionary of MNE objects, structured as {subject: {condition: MNE epoch objects}}
    - condition_names: List of condition names.
    - rois: List of region of interest (ROI) names.
    - subjects: List of subjects.
    - sig_electrodes_per_subject_roi: Dictionary that maps ROIs to subjects and their corresponding electrodes.
    
    Returns:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI. 
                          Each LabeledArray concatenates the subject data along the channels dimension.
                          Dimensions of LabeledArray: [Conditions, Electrodes (concatenated), Trials, Timepoints]
    """
    roi_labeled_arrays = {}

    # Loop through each ROI
    for roi in rois:
        roi_labeled_array = None

        # Find the maximum number of trials for each condition across all subjects
        max_trials_per_condition = {condition: 0 for condition in condition_names}

        # First pass to find the max number of trials per condition
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
            if not sig_electrodes:
                continue
            for condition_name in condition_names:
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                epochs_data = epochs.get_data(copy=True)  # Shape: (trials, channels, timepoints)
                max_trials_per_condition[condition_name] = max(max_trials_per_condition[condition_name], epochs_data.shape[0])

        # Second pass to gather data for each subject and condition
        for sub in subjects:
            sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
            if not sig_electrodes:
                continue

            subject_nested_dict = {}

            # Get channel names for this subject's ROI
            sub_channel_names = [f"{sub}-{sig_electrode}" for sig_electrode in sig_electrodes]

            # Loop through each condition
            for condition_name in condition_names:
                # Extract the epoch data for the current condition and subject, pick only the electrodes of interest
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
                epochs_data = epochs.get_data(copy=True)  # Shape: (trials, channels, timepoints)
                # Get the number of trials and the target number of trials
                n_trials, n_channels, n_timepoints = epochs_data.shape
                max_trials = max_trials_per_condition[condition_name]

                # Pad with NaNs if necessary
                if n_trials < max_trials:
                    # Create a padded array with NaNs
                    padded_data = np.full((max_trials, n_channels, n_timepoints), np.nan)
                    padded_data[:n_trials, :, :] = epochs_data  # Copy existing data into the padded array
                else:
                    padded_data = epochs_data

                subject_nested_dict[condition_name] = padded_data
            times = epochs.times # same times across conditions, so just grab times of last condition
            str_times = [str(time) for time in times]
            np_array_str_times = np.array(str_times) # make this a numpy array of strings so it plays nice with aaron's code

            # Create a LabeledArray for the subject
            subject_labeled_array = LabeledArray.from_dict(subject_nested_dict)
            subject_labeled_array.labels[2] = sub_channel_names  # Add channel labels for this subject
            subject_labeled_array.labels[3] = np_array_str_times # Add time labels for this subject. This breaks things. But why????? 9/10. 
            
            if roi_labeled_array is None:
                roi_labeled_array = subject_labeled_array
            else:
                roi_labeled_array = roi_labeled_array.concatenate(subject_labeled_array, axis=2) # concatenate along channels axis

        # Add the concatenated LabeledArray to the ROI dictionary
        roi_labeled_arrays[roi] = roi_labeled_array

    return roi_labeled_arrays


In [151]:
# Example call to the function
roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

In [157]:
roi_labeled_arrays['lpfc']['Stimulus/c25.0/r25.0']

array([[[-1.95866501, -1.99797666, -1.99603009, ...,  0.52920264,
          0.28267011,  0.07571847],
        [-0.36993843, -0.46310261, -0.59090614, ...,  0.40243363,
          0.45360208,  0.47372442],
        [ 0.09367087,  0.05117829,  0.03652748, ..., -1.62141681,
         -1.6488266 , -1.65550506],
        ...,
        [ 0.4270342 ,  0.56077248,  0.63189501, ..., -1.17867291,
         -1.13246894, -1.08277845],
        [-0.55124217, -0.55391127, -0.58436954, ...,  0.37085962,
          0.21903849,  0.05355533],
        [ 2.7285645 ,  2.82342935,  2.88325024, ..., -0.89726233,
         -0.82099926, -0.7525385 ]],

       [[ 0.72307342,  0.82587808,  0.90708303, ...,  1.13180387,
          1.06835997,  1.02024138],
        [-0.67485756, -0.61215526, -0.56731725, ..., -0.96563709,
         -0.83458418, -0.67207652],
        [-1.37012219, -1.33064234, -1.2719003 , ..., -0.49311006,
         -0.71884269, -0.94139785],
        ...,
        [ 0.66525507,  0.49917999,  0.32196042, ...,  

set up time bins, time indexing, array reshaping, and decoding functions 9/12

In [120]:
def create_time_bins(time_points, bin_size=0.2):
    """
    Create time bins from the given time points array.

    Parameters:
    - time_points: 1D numpy array of time points.
    - bin_size: The size of each time bin in seconds (default is 0.1s, which is 100 ms).

    Returns:
    - bins: A list of numpy arrays, where each array contains the time points within that bin.
    """
    bins = []
    start_time = np.min(time_points)
    end_time = np.max(time_points)
    current_time = start_time
    
    while current_time < end_time:
        next_time = current_time + bin_size
        if next_time > end_time:
            next_time = end_time
        bins.append(time_points[(time_points >= current_time) & (time_points < next_time)])
        current_time = next_time

    # Ensure that the last bin includes the endpoint
    if time_points[-1] not in bins[-1]:
        bins[-1] = np.append(bins[-1], time_points[-1])

    return bins

def get_bin_centers(time_bins):
    """
    Calculate the center of each time bin.

    Parameters:
    - time_bins: List of numpy arrays, where each array represents a time bin.

    Returns:
    - bin_centers: A list of bin center values (floats).
    """
    bin_centers = [np.mean(bin) for bin in time_bins]
    return bin_centers


In [121]:
def index_timepoints_by_bins(labeled_array, time_bins):
    """
    Index time points from a LabeledArray using given time bins.

    Parameters:
    - labeled_array: The LabeledArray from which to extract timepoints (e.g., roi_labeled_arrays['lpfc']).
    - time_bins: List of numpy arrays, where each array represents a time bin.

    Returns:
    - bin_indices: A list of lists, where each list contains the indices of the time points that fall into the corresponding bin.
    """
    # Get the time points from the LabeledArray
    time_points = labeled_array.labels[3].astype(float)  # Convert the time labels to floats

    bin_indices = []
    
    # Loop over the time bins and find the corresponding indices
    for time_bin in time_bins:
        indices_in_bin = np.where(np.isin(time_points, time_bin))[0]  # Get indices of time points in this bin
        bin_indices.append(indices_in_bin)
    
    return bin_indices

hm to use mixup i think i shouldn't remove nan values in reshape_labeled_array_for_decoding. But then how to concatenate trials? Actually i think might be fine to just not do the nan removal step. Try this tonight 9/23.

In [161]:
def reshape_labeled_array_for_decoding(labeled_array):
    """
    Reshape a single LabeledArray, removing trials that have NaN values and reshaping to be trials by channels*timepoints.

    Parameters:
    - labeled_array: A LabeledArray where the 0th axis labels are conditions, 1st axis labels are trials, 
                     2nd axis labels are channels, and 3rd axis labels are timepoints.

    Returns:
    - reshaped_data: Dictionary where keys are condition names and values are reshaped arrays (trials x channels*timepoints).
    """
    reshaped_labeled_array = {}
    
    # Extract the condition labels (which are in labeled_array.labels[0])
    condition_names = labeled_array.labels[0]

    # Loop over each condition
    for condition_name in condition_names:
        # Extract the data for the current condition
        condition_data = labeled_array[condition_name]
        
        # # Find the indices of trials that do not contain NaNs
        # valid_trial_indices = ~np.isnan(condition_data).any(axis=(1, 2))

        # # Use the take method to select only the valid trials
        # condition_data_clean = condition_data.take(indices=np.where(valid_trial_indices)[0], axis=0)
        
        # # Combine the channels and timepoints axes (axis 1 and axis 2) into a single axis
        # channel_time_combined_array = condition_data_clean.reshape(condition_data_clean.shape[0], -1)
        
        # combine channels and timepoints axes into a single axis, but include all nan trials too. Hm should I get rid of trial outliers though? Those are NaN for all channels right..? Instead of just some?
        channel_time_combined_array = condition_data.reshape(condition_data.shape[0], -1)
        
        # Store the processed data for this condition
        reshaped_labeled_array[condition_name] = channel_time_combined_array

    return reshaped_labeled_array

def process_all_rois_for_decoding(roi_labeled_arrays):
    """
    Loop through all ROIs and apply reshaping function to each LabeledArray.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.

    Returns:
    - all_processed_data: Dictionary where keys are ROIs and values are dictionaries of reshaped data for each condition.
    """
    reshaped_roi_labeled_arrays_for_decoding = {}

    # Loop through each ROI in the dictionary
    for roi, labeled_array in roi_labeled_arrays.items():
        # Apply the reshaping function to the current labeled array
        reshaped_labeled_array = reshape_labeled_array_for_decoding(labeled_array)

        # Store the reshaped data for this ROI
        reshaped_roi_labeled_arrays_for_decoding[roi] = reshaped_labeled_array

    return reshaped_roi_labeled_arrays_for_decoding


In [162]:
def concatenate_conditions_by_string(roi_labeled_arrays, roi, strings_to_find):
    """
    Concatenate trials across condition names that contain specific strings.
    Assign labels based on the groupings of the conditions.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
    - roi: The specific ROI to process.
    - strings_to_find: List of strings or list of lists of strings to search for in condition names.
                      If a list of strings is provided, each string is treated as its own condition group.
                      If a list of lists is provided, each sublist represents a group of conditions.

    Returns:
    - concatenated_data: The concatenated trials by channels*timepoints across the matching conditions.
    - labels: A numpy array of labels (0, 1, 2, ...) corresponding to each group of conditions.
    - cats: Dictionary of {condition_name: index} for decoding.
    """
    concatenated_data = []
    labels = []
    cats = {}

    # Track current label index
    current_label = 0

    # Normalize strings_to_find so each entry is a list (whether it's a string or a list of strings)
    if isinstance(strings_to_find, list) and all(isinstance(s, str) for s in strings_to_find):
        # If it's a flat list of strings, convert each string into its own single-item list
        strings_to_find = [[s] for s in strings_to_find]

    # Iterate over each group (whether it's a single string or a list of strings)
    for string_group in strings_to_find:
        # Find condition names that match any of the strings in the current string_group
        matching_conditions = [cond for cond in roi_labeled_arrays[roi].keys() if any(s in cond for s in string_group)]

        if not matching_conditions:
            continue

        # Concatenate data for all matching conditions
        data_to_concatenate = []
        for cond in matching_conditions:
            # Extract data for the current condition
            data = roi_labeled_arrays[roi][cond]  # Shape: (trials, channels, timepoints)
            data_to_concatenate.append(data)
            
            # Update labels for the current condition group
            labels.extend([current_label] * data.shape[0])
        
        # Check if we have data to concatenate for this condition group
        if data_to_concatenate:
            concatenated_data.append(np.concatenate(data_to_concatenate, axis=0))

        # Assign current label to the condition group (based on the first string in the group for reference)
        cats[tuple(string_group)] = current_label
        current_label += 1

    # Ensure there is data to concatenate
    if not concatenated_data:
        raise ValueError(f"No matching conditions found for ROI: {roi} and strings: {strings_to_find}")

    # Concatenate all condition data along the trials axis
    concatenated_data = np.concatenate(concatenated_data, axis=0)
    
    return concatenated_data, np.array(labels), cats

set up reshaped arrays for each time bin 9/12

In [163]:
# Assuming temporal_datasets is a dictionary
first_sub = next(iter(subjects_mne_objects.keys()))
first_cond = next(iter(subjects_mne_objects[first_sub].keys()))

time_points = subjects_mne_objects[first_sub][first_cond]['HG_ev1_rescaled'].times

# Create time bins
bin_size = 0.2
time_bins = create_time_bins(time_points, bin_size=bin_size)

In [164]:
class TimeBinnedROILabeledArray:
    def __init__(self, bin_center, bin_size, indices, data):
        """
        Initialize a TimeBinnedROILabeledArray object.

        Parameters:
        - bin_center: The center of the time bin.
        - indices: The indices of the time points that belong to this bin.
        - data: The reshaped ROI labeled arrays for decoding.
        """
        self.bin_center = bin_center
        self.indices = indices
        self.data = data
        self.bin_size = bin_size
        self.bin_start = bin_center - bin_size/2
        self.bin_end = bin_center + bin_size/2
        
class TimeBinnedArraysDataManager:
    def __init__(self, roi_labeled_arrays, time_bins, bin_size, rois):
        """
        Initialize the class to manage all time-binned reshaped ROI labeled arrays.

        Parameters:
        - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
        - time_bins: The time bins generated for the data.
        - rois: The regions of interest to process.
        """
        self.roi_labeled_arrays = roi_labeled_arrays
        self.time_bins = time_bins
        self.rois = rois
        self.bin_centers = get_bin_centers(time_bins)
        self.bin_size = bin_size
        self.bin_indices = None
        self.binned_data = []  # List of TimeBinnedROILabeledArray objects

    def index_time_bins(self):
        """Index the time points for each bin."""
        first_roi = next(iter(self.roi_labeled_arrays.keys()))
        self.bin_indices = index_timepoints_by_bins(self.roi_labeled_arrays[first_roi], self.time_bins)

    def process_bins(self):
        """Process each time bin and reshape the ROI labeled arrays for decoding."""
        self.index_time_bins()  # Ensure bin indexing is done before processing bins

        for i, indices in enumerate(self.bin_indices):
            bin_center = self.bin_centers[i]
            roi_labeled_arrays_this_bin = {}

            for roi in self.rois:
                # Index the ROI labeled array and grab just these time indices
                roi_labeled_arrays_this_bin[roi] = self.roi_labeled_arrays[roi][:, :, :, indices]

            # Drop trial NaNs and concatenate channel-time for the time-indexed arrays
            reshaped_roi_labeled_arrays_for_decoding_this_bin = process_all_rois_for_decoding(roi_labeled_arrays_this_bin)

            # Create a TimeBinnedROILabeledArray object and store it
            self.binned_data.append(TimeBinnedROILabeledArray(bin_center, bin_size, indices, reshaped_roi_labeled_arrays_for_decoding_this_bin))

    def get_binned_data(self):
        """Return the list of TimeBinnedROILabeledArray objects."""
        return self.binned_data

# make the wrapper class first with all the attributes, then get the list of individual time binned arrays. Name better?
TimeBinnedArraysDataManager = TimeBinnedArraysDataManager(roi_labeled_arrays, time_bins, bin_size, rois)
TimeBinnedArraysDataManager.process_bins()
time_binned_arrays = TimeBinnedArraysDataManager.get_binned_data()

# Print the first bin's center for example
time_binned_arrays[0].data


{'lpfc': {'Stimulus/i25.0/s25.0': array([[-0.73826939, -0.79218125, -0.86250168, ...,  0.16549054,
           0.07402185, -0.01390438],
         [ 0.74493533,  0.68340832,  0.61799419, ...,  1.02985811,
           1.00662029,  1.01148868],
         [-1.43706226, -1.57506466, -1.7232815 , ...,  0.38158751,
           0.33548358,  0.26592141],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]])
  labels(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',

set up decoding output directory and conditions to compare 9/12

In [165]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'decoding', 'figs')

if conditions == stimulus_experiment_conditions:
    strings_to_find = [['c25.0', 'c75.0'], ['i25.0', 'i75.0']]  # Strings to search for in condition names
    condition_0_name = 'congruent' # this is for plot names
    condition_1_name = 'incongruent'
elif conditions == stimulus_conditions:
    strings_to_find = ['Taskg', 'Taskl']
    condition_0_name = 'Taskg'
    condition_1_name = 'Taskl'

do decoding on the reshaped time indexed arrays 9/12

let's try a class for storing and plotting the decoding scores for each roi 9/13

In [166]:
class ROIScoreData:
    def __init__(self, roi, bin_size):
        self.roi = roi
        self.bin_size = bin_size
        self.bin_centers = []
        self.accuracy_scores = []
        self.precision_scores = []
        self.recall_scores = []
        self.f1_scores = []
        self.d_prime_scores = []

    def add_scores(self, bin_center, accuracy, precision, recall, f1, d_prime):
        self.bin_centers.append(bin_center)
        self.accuracy_scores.append(accuracy)
        self.precision_scores.append(precision)
        self.recall_scores.append(recall)
        self.f1_scores.append(f1)
        self.d_prime_scores.append(d_prime)

    def plot_scores(self, strings_to_find, save_dir):
        plt.figure(figsize=(10, 6))

        # Plot accuracy
        plt.plot(self.bin_centers, self.accuracy_scores, label='Accuracy', marker='o')

        # # Plot precision
        # plt.plot(self.bin_centers, self.precision_scores, label='Precision', marker='o')

        # # Plot recall
        # plt.plot(self.bin_centers, self.recall_scores, label='Recall', marker='o')

        # # Plot f1 score
        # plt.plot(self.bin_centers, self.f1_scores, label='F1 Score', marker='o')

        # Plot d-prime
        plt.plot(self.bin_centers, self.d_prime_scores, label='D-Prime', marker='o')

        # Set plot details
        plt.xlabel('Time Bin Centers (s)')
        plt.ylabel('Score')
        plt.title(f'{condition_0_name} vs. {condition_1_name} Decoding Over Time for ROI: {self.roi}')
        plt.legend()
        plt.grid(True)
        plt.ylim(0,1)

        # Save the figure
        file_name = f'{self.roi}_{condition_0_name}_vs_{condition_1_name}_decoding_over_time_dprime_acc.png'
        plt.savefig(os.path.join(save_dir, file_name))
        plt.close()


In [167]:
# Initialize a dictionary to hold the ROIScoreData objects
roi_score_data = {}

# Process each time bin and region of interest
for roi in rois:
    # Initialize the ROIScoreData object for the current ROI
    roi_scores = ROIScoreData(roi, bin_size)
    
    for time_binned_array in time_binned_arrays:

        # Concatenate the trials and get labels
        concatenated_data, labels, cats = concatenate_conditions_by_string(time_binned_array.data, roi, strings_to_find)
        print(concatenated_data.shape)
        
        # Iterate over the groups in strings_to_find
        for string_group in strings_to_find:
            # If it's a single string (old behavior)
            if isinstance(string_group, str):
                # Count trials for the single condition group
                num_trials = len([label for label in labels if label == cats[(string_group,)]])
                print(f"{string_group} has {num_trials} trials")
            
            # If it's a list of strings (new behavior)
            elif isinstance(string_group, list):
                # Count trials for the group of strings (e.g., ['i25.0', 'i75.0'])
                num_trials = len([label for label in labels if label == cats[tuple(string_group)]])
                print(f"{string_group} has {num_trials} trials")

        # Create a Decoder and run cross-validation
        decoder = Decoder(cats, 0.80, oversample=True, n_splits=5, n_repeats=5)

        # Use the concatenated data for the decoder and get scores. Use normalize of none or else the scores will be skewed.
        cm_avg, scores = decoder.cv_cm_return_scores(concatenated_data, labels, normalize=None)

        # Store the decoding scores and bin center for plotting
        bin_center = time_binned_array.bin_center  # Get the bin center for x-axis

        # Add scores to the ROIScoreData object
        roi_scores.add_scores(
            bin_center,
            scores['accuracy'],
            np.mean(scores['precision']),  # Average across classes if multi-class
            np.mean(scores['recall']),     # Average across classes
            np.mean(scores['f1']),         # Average across classes
            np.mean(scores['d_prime'])     # Average across classes
        )

    # Save the scores for this ROI
    roi_score_data[roi] = roi_scores

# Plot the decoding scores for each ROI
for roi, roi_scores in roi_score_data.items():
    roi_scores.plot_scores(strings_to_find, save_dir)

(1024, 2392)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2392)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2392)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 2346)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
(1024, 1196)
['c25.0', 'c75.

do decoding across all timepoints (hm refactor this to use the ROILabeledArray objects maybe 9/14)

In [168]:
def get_and_plot_confusion_matrix_for_rois(roi_labeled_arrays, rois, strings_to_find, condition_0_name, condition_1_name, save_dir, time_interval_name=None, n_splits=5, n_repeats=5):
    """
    Compute the confusion matrix for each ROI and return it. Does this using all time points for training.

    Parameters:
    - roi_labeled_arrays: Dictionary containing the reshaped data for each ROI.
    - rois: List of regions of interest (ROIs) to process.
    - strings_to_find: List of strings or string groups to identify conditions.
    - condition_0_name: Name for condition 0 (used for labeling).
    - condition_1_name: Name for condition 1 (used for labeling).
    - save_dir: Directory to save the confusion matrix plots.
    - time_interval_name: Optional string to add to the filename for the time interval (e.g., 'pre_stimulus' or 'post_stimulus').

    Returns:
    - confusion_matrices: Dictionary containing confusion matrices for each ROI.
    """
    confusion_matrices = {}

    reshaped_roi_labeled_arrays_for_decoding = process_all_rois_for_decoding(roi_labeled_arrays)

    for roi in rois:
        print(f"Processing ROI: {roi}")
        
        # Concatenate the trials and get labels
        concatenated_data, labels, cats = concatenate_conditions_by_string(reshaped_roi_labeled_arrays_for_decoding, roi, strings_to_find)
        print(f"Concatenated data shape for {roi}: {concatenated_data.shape}")
        
        # Iterate over the groups in strings_to_find
        for string_group in strings_to_find:
            # If it's a single string (old behavior)
            if isinstance(string_group, str):
                # Count trials for the single condition group
                num_trials = len([label for label in labels if label == cats[(string_group,)]])
                print(f"{string_group} has {num_trials} trials")
            
            # If it's a list of strings (new behavior)
            elif isinstance(string_group, list):
                # Count trials for the group of strings (e.g., ['i25.0', 'i75.0'])
                num_trials = len([label for label in labels if label == cats[tuple(string_group)]])
                print(f"{string_group} has {num_trials} trials")

        # Create a Decoder and run cross-validation
        decoder = Decoder(cats, 0.80, oversample=True, n_splits=n_splits, n_repeats=n_repeats)

        # Use the concatenated data for the decoder
        cm = decoder.cv_cm(concatenated_data, labels, normalize='true')
        cm_avg = np.mean(cm, axis=0)

        # Store the confusion matrix in the dictionary
        confusion_matrices[roi] = cm_avg

        # Convert tuple labels to simple strings for display
        display_labels = [
            key[0] if isinstance(key, tuple) and len(key) == 1 else str(key)
            for key in cats.keys()
        ]
        
        # Plot the Confusion Matrix
        fig, ax = plt.subplots()
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_avg, display_labels=display_labels)
        disp.plot(ax=ax)

        # Save the figure with the time interval in the filename
        time_str = f"_{time_interval_name}" if time_interval_name else ""
        file_name = f'{roi}_{condition_0_name}_vs_{condition_1_name}{time_str}_time_averaged_confusion_matrix_{n_splits}splits_{n_repeats}repeats.png'
        plt.savefig(os.path.join(save_dir, file_name))
        plt.close()

    return confusion_matrices


In [169]:
# Call the new function with the appropriate parameters
confusion_matrices = get_and_plot_confusion_matrix_for_rois(
    roi_labeled_arrays=roi_labeled_arrays,
    rois=rois,
    strings_to_find=strings_to_find,
    condition_0_name=condition_0_name,
    condition_1_name=condition_1_name,
    save_dir=save_dir,
    time_interval_name=None,  # If not dealing with specific time intervals
    n_splits=5,
    n_repeats=10
)

Processing ROI: lpfc
Concatenated data shape for lpfc: (1024, 29486)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
Processing ROI: v1
Concatenated data shape for v1: (1008, 7692)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ
Concatenated data shape for occ: (1008, 26281)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (1008, 14743)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ_best_filtered
Concatenated data shape for occ_best_filtered: (1008, 6410)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials


In [139]:
# replaced with a function 9/21
# reshaped_roi_labeled_arrays_for_decoding = process_all_rois_for_decoding(roi_labeled_arrays)

# for roi in rois:
#     print(roi)
    
#     # Concatenate the trials and get labels
#     concatenated_data, labels, cats = concatenate_conditions_by_string(reshaped_roi_labeled_arrays_for_decoding, roi, strings_to_find)
#     print(concatenated_data.shape)
    
#     # Iterate over the groups in strings_to_find
#     for string_group in strings_to_find:
#         # If it's a single string (old behavior)
#         if isinstance(string_group, str):
#             # Count trials for the single condition group
#             num_trials = len([label for label in labels if label == cats[(string_group,)]])
#             print(f"{string_group} has {num_trials} trials")
        
#         # If it's a list of strings (new behavior)
#         elif isinstance(string_group, list):
#             # Count trials for the group of strings (e.g., ['i25.0', 'i75.0'])
#             num_trials = len([label for label in labels if label == cats[tuple(string_group)]])
#             print(f"{string_group} has {num_trials} trials")

#     # Create a Decoder and run cross-validation
#     decoder = Decoder(cats, 0.80, oversample=True, n_splits=5, n_repeats=5)

#     # Use the concatenated data for the decoder
#     cm = decoder.cv_cm(concatenated_data, labels, normalize='true')
#     cm = np.mean(cm, axis=0)

#     # Plot the Confusion Matrix
#     fig, ax = plt.subplots()
#     disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cats.keys())
#     disp.plot(ax=ax)

#     # Save the figure with the strings_to_find reflected in the filename
#     file_name = f'{roi}_{condition_0_name}_vs_{condition_1_name}_time_averaged_confusion_matrix.png'
#     plt.savefig(os.path.join(save_dir, file_name))
#     plt.close()

prestimulus vs poststimulus confusion matrices 9/21

In [170]:
def get_data_in_time_range(labeled_array, time_range):
    """
    Extract data from a LabeledArray where the time points fall within a given range, using the LabeledArray `take` function.
    
    Parameters:
    - labeled_array: The LabeledArray containing time points as labels.
    - time_range: A tuple (start_time, end_time) representing the range of time.
    
    Returns:
    - filtered_data: A LabeledArray containing only the data where the timepoints are within the specified range.
    """
    start_time, end_time = time_range

    # Assume that the time labels are stored in the 4th dimension (index 3)
    time_points = np.array(labeled_array.labels[3], dtype=float)  # convert to floats, ensure conversion to float numpy array

    # Find the indices of time points within the specified range
    time_indices = np.where((time_points >= start_time) & (time_points <= end_time))[0]

    # Use the take function to select the time indices along the time axis (axis=3)
    filtered_data = labeled_array.take(time_indices, axis=3)

    return filtered_data


In [171]:
# Define the time intervals of interest
pre_stimulus_interval = (-1.0, 0.0)  # Pre-stimulus: time -1 to 0 seconds
post_stimulus_interval = (0.0, 1.5)  # Post-stimulus: time 0 to 1.5 seconds
pre_stimulus_roi_labeled_arrays = {}
post_stimulus_roi_labeled_arrays = {}

for roi, labeled_array in roi_labeled_arrays.items():
    pre_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, pre_stimulus_interval)
    post_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, post_stimulus_interval)


pre_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois(
    roi_labeled_arrays=pre_stimulus_roi_labeled_arrays,
    rois=rois,
    strings_to_find=strings_to_find,
    condition_0_name=condition_0_name,
    condition_1_name=condition_1_name,
    save_dir=save_dir,
    time_interval_name='pre_stimulus',  # If not dealing with specific time intervals
    n_splits=5,
    n_repeats=10
)

post_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois(
    roi_labeled_arrays=post_stimulus_roi_labeled_arrays,
    rois=rois,
    strings_to_find=strings_to_find,
    condition_0_name=condition_0_name,
    condition_1_name=condition_1_name,
    save_dir=save_dir,
    time_interval_name='post_stimulus',  # If not dealing with specific time intervals
    n_splits=5,
    n_repeats=10
)

Processing ROI: lpfc
Concatenated data shape for lpfc: (1024, 11822)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
Processing ROI: v1
Concatenated data shape for v1: (1008, 3084)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ
Concatenated data shape for occ: (1008, 10537)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (1008, 5911)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ_best_filtered
Concatenated data shape for occ_best_filtered: (1008, 2570)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials
Processing ROI: lpfc
Concatenated data shape for lpfc: (1024, 17710)
['c25.0', 'c75.0'] has 512 trials
['i25.0', 'i75.0'] has 512 trials
Processing ROI: v1
Concatenated data shape for v1: (1008, 4620)
['c25.0', 'c75.0'] has 504 trials
['i25.0', 'i75.0'] has 504 trials
Processing RO

define colors for plotting (not used yet as of 8/21)

In [105]:
# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'congruency_congruency_proportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'switch_type_switch_proportion': 'gray',
    'bigLetter': 'green',
    'big_letter': 'green',
    'smallLetter': 'orange',
    'small_letter': 'orange',
    'task': 'gray',
    'c75.0': 'pink',
    'i75.0': 'pink',
    'c25.0': 'gold',
    'i25.0': 'gold',
    'r25.0': 'lightblue',
    's25.0': 'lightblue',
    'r75.0': 'purple',
    's75.0': 'purple'
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'BigLetters': '-',

    'big letter H': '--',
    'BigLetterh': '--',

    'small letter S': '-',
    'SmallLetters': '-',

    'small letter H': '--',
    'SmallLetterh': '--',

    'task G': '-',
    'Taskg': '-',

    'task L': '--',
    'Taskl': '--',

    'congruent': '-',
    'c': '-',

    'incongruent': '--',
    'i': '--',

    'repeat': '-',
    'r': '-',

    'switch': '--',
    's': '--',

    'c25.0': '-',
    'c75.0': '-',
    'i25.0': '--',
    'i75.0': '--'
}